# New features in Imviz, May 2024
Brett Morris

### JWebbinar 2024 on jdaviz

jdaviz:
* [Documentation](https://jdaviz.readthedocs.io/)
* [Source code](https://github.com/spacetelescope/jdaviz)

### Outline

This tutorial will demonstrate features recently introduced (since 2023) to Imviz ([docs](https://jdaviz.readthedocs.io/en/latest/imviz/index.html)), the jdaviz configuration for 2D images. These new features include: 
* Introducing the `Orientation` plugin, enabling image rotation
* Introducing the `Footprints` plugin, visualizing the fields of view of JWST instruments and beyond.
* Introducing convenience methods for automatic RGB color composite visualizations in the `Plot Options` plugin.
* Introducing the `Data Quality` plugin for visualizing data quality bit flags on Level 2 data products for JWST, Roman, and HST.

### Loading a reference image, launching Imviz

In [ ]:
import warnings

import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy import units as u
from astropy.utils.data import get_pkg_data_filename
from astroquery.mast import Observations

from jdaviz import Imviz

Here we load an example image of the Horsehead Nebula from the STScI Digitized Sky Survey (DSS). This is a FITS image that's included in your installation of `astropy`. (It's actually derived from a photographic plate!)

In [ ]:
filename = get_pkg_data_filename('tutorials/FITS-images/HorseHead.fits')

We initialize `Imviz` and load the DSS image like so:

In [ ]:
imviz = Imviz()
imviz.load_data(filename, data_label='STScI DSS')
imviz.show()

By default, any images loaded in the future will be "matched by pixel," such that the (0, 0) pixel of each image overlaps. We can show the images matched by their WCS in the `Orientation` plugin like so:

In [ ]:
orientation = imviz.plugins['Orientation']
orientation.link_type = "WCS"

### Tweaking plot options for clearer visualization

Let's adjust some of the `Plot Options` parameters to make this image a bit clearer in `Imviz`:

In [ ]:
plot_options = imviz.plugins['Plot Options']

plot_options.image_color_mode = 'Monochromatic'
plot_options.image_opacity = 1
plot_options.image_color = '#ffffff'  # white
plot_options.stretch_function = 'Linear'
plot_options.stretch_preset = '99.5%'

### Reorienting the viewer

This FITS image was generated with its pixel grid aligned to the standard "North-up East-left" orientation, but you wouldn't see a press release in this orientation. They call this the "Horsehead Nebula," but that's clearer in a certain orientation. Let's rotate the field of view in Imviz by 90 degrees counter-clockwise:

In [ ]:
orientation.rotation_angle = 90
orientation.add_orientation()

### Visualize an instrument footprint

Now suppose we want to plan an observing program for short wavelength JWST [NIRCam](https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-observing-modes/nircam-imaging) imaging of the peak of the horse's mane. We can visualize the footprint of JWST instruments in the `Footprints` plugin, and adjust the RA, Dec, and position angle to capture the features we are most interested in.

In [ ]:
footprints = imviz.plugins['Footprints']
footprints.preset = 'NIRCam:short'
footprints.keep_active = True
footprints.ra = 85.2105
footprints.dec = -2.4575
footprints.pa = 68.2882

### Load JWST NIRCam observations

Congratulations, your GO program to observe the horse's mane was approved, the observations were successful, and we're ready to load in the new observations! 

In [ ]:
labels = [
    "F070W",
    "F187N",
    "F212N"
]
uris = [
    "mast:jwst/product/jw01192-o015_t015_nircam_clear-f070w_i2d.fits",
    "mast:jwst/product/jw01192-o015_t015_nircam_clear-f187n_i2d.fits",
    "mast:jwst/product/jw01192-o015_t015_nircam_clear-f212n_i2d.fits",
]

data_dir = 'staged_fits_webbinar_2024/'

for uri, data_label in zip(uris, labels):
    fn = uri.split('/')[-1]
    result = Observations.download_file(uri, local_path=f'{data_dir}/{fn}')
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz.load_data(f'{data_dir}/{fn}', data_label=data_label)

Let's toggle the visibility of the DSS image, now that we have our glorious JWST images. 

In [ ]:
plot_options.layer = 'STScI DSS[PRIMARY,1]'  # this is the first-loaded image
plot_options.image_visible = False

### Quick color composites by assigning RGB channels

We currently have three images in the viewer from observations at: 0.7, 1.87, and 2.12 µm. We can quickly turn this into an RGB like so:

In [ ]:
plot_options.apply_RGB_presets()

We can continue to fine-tune our plot options to visualize the features that are useful for your science:

In [ ]:
for layer in plot_options.layer.choices:
    plot_options.layer = layer
    plot_options.stretch_function = 'linear'
    plot_options.stretch_preset = '99%'
    plot_options.image_opacity = 0.8

### Inspecting data quality

Now suppose there's a feature in your images that you don't fully understand. You might want to confirm that a potentially interesting feature is real, and not an artifact of the `jwst` reduction pipeline. Sometimes inspecting lower level data products is useful for this task. The NIRCam images above are Level 3 products, but in this next example, we're going to use a Level 2 image that contains a Data Quality (DQ) array, flagging possible problems encountered by the `jwst` pipeline. We will use observations of the Helix Nebula (NGC 7293). First let's download the data:

In [ ]:
uri = 'mast:JWST/product/jw02211044001_02201_00002_nrcblong_cal.fits'
fn = uri.split('/')[-1]
path = f'{data_dir}/{fn}'
result = Observations.download_file(uri, local_path=path)

Now we will load the data into `Imviz`. This time we will specifically load two FITS extensions: the science data labeled `SCI`, and the data quality array labeled `DQ`. We will also open the Data Quality plugin:

In [ ]:
imviz = Imviz()
imviz.load_data(path, ext=('SCI', 'DQ'))
imviz.show()

dq = imviz.plugins['Data Quality']
dq.open_in_tray()